In [2]:
import re
import praw
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import requests  

import os
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
client_id = os.getenv('CLIENT_ID')
client_secret = os.getenv('CLIENT_SECRET')
user_agent = os.getenv('USER_AGENT')
reddit = praw.Reddit(client_id = client_id, 
                    client_secret =client_secret, 
                    user_agent=user_agent)

In [4]:
posts = []
#change subreddit to specific page
sr = reddit.subreddit('AmITheAsshole')

In [5]:
posts = []

for post in sr.new(limit=1000):
    posts.append([post.created, post.url, post.id, post.score, post.num_comments, post.author, post.link_flair_text, post.title, post.selftext])

In [6]:
posts = []
last_post = None 

for _ in range(10): 
    new_posts = sr.new(limit=1000, params={'after': last_post})
    batch = [
        [post.created, post.url, post.id, post.score, post.num_comments,
         post.author, post.link_flair_text, post.title, post.selftext]
        for post in new_posts
    ]
    if not batch:
        break
    posts.extend(batch)
    last_post = batch[-1][2]
    

In [7]:
len(posts)


8792

In [8]:
posts = pd.DataFrame(posts, columns=['created', 'url', 'id', 'score', 'num_comments', 'author', 'flair', 'title', 'body'])
# posts['created'] = pd.to_datetime(posts['created'], unit='s')


In [9]:
display(posts.head())

,created,url,id,score,num_comments,author,flair,title,body
0,1.744569e+09,https://www.reddit.com/r/AmItheAsshole/comment...,1jyelws,1,2,Ilovemydog2025,None,AITA for not giving our dog back to my ex afte...,"Eight months ago, my ex (30F) broke up with me..."
1,1.744569e+09,https://www.reddit.com/r/AmItheAsshole/comment...,1jyei67,5,7,BeththeBethBeth,None,AITA because I wouldn't put my step dad on my ...,"\n\nAITA, \nSo my mother and father broke up ..."
2,1.744569e+09,https://www.reddit.com/r/AmItheAsshole/comment...,1jyedz4,1,3,HML___,None,AITA for not apologising to my dad,I (17F) have been arguing a lot with my parent...
3,1.744568e+09,https://www.reddit.com/r/AmItheAsshole/comment...,1jye91z,0,6,Good_Researcher_6342,None,AITA for not locking the door and ending up ge...,"Okay, so this is super embarrassing, but I nee..."
4,1.744567e+09,https://www.reddit.com/r/AmItheAsshole/comment...,1jydszy,6,17,Ok_Champion_2743,None,AITA because i got angry to my dad because he ...,so my mom mocked me as a child when i told the...


In [10]:
df = posts

In [11]:
df['flair'].unique()

array([None, 'POO Mode Activated 💩', 'Not the A-hole', 'Asshole',
       'No A-holes here', 'Everyone Sucks', 'TL;DR', 'Not enough info',
       'UPDATE', 'Asshole POO Mode', '', 'Not the A-hole POO Mode',
       'Everyone Sucks POO Mode', 'Open Forum'], dtype=object)

In [12]:
df = df[df['flair'].isin(['Asshole', 'Not the A-hole', 'No A-holes here',
       'Not enough info', 'Everyone Sucks', ''])]
df['flair'].unique()

array(['Not the A-hole', 'Asshole', 'No A-holes here', 'Everyone Sucks',
       'Not enough info', ''], dtype=object)

In [13]:
df.size
df.shape


(7780, 9)

In [14]:
display(df.head())

,created,url,id,score,num_comments,author,flair,title,body
56,1.744505e+09,https://www.reddit.com/r/AmItheAsshole/comment...,1jxvssj,22,11,Theotherthirdsun,Not the A-hole,AITA for not inviting one friend on our trip?,I've been friends with “Anna” and “Juan” since...
57,1.744505e+09,https://www.reddit.com/r/AmItheAsshole/comment...,1jxvoyx,901,126,shiny-baby-cheetah,Not the A-hole,"AITA for the joke I made at a get together, in...",Istg I've got to stop giving my weekends away ...
58,1.744500e+09,https://www.reddit.com/r/AmItheAsshole/comment...,1jxu9hu,80,45,Boring_Design_6445,Not the A-hole,AITA for not turning on Life 360 for my mom,I(19f) am a university student and I still li...
59,1.744499e+09,https://www.reddit.com/r/AmItheAsshole/comment...,1jxtxfl,0,35,Embarrassed_Data4018,Asshole,AITA for bringing our daughter to my husband’s...,\n\nAITA for bringing our daughter to my husba...
60,1.744497e+09,https://www.reddit.com/r/AmItheAsshole/comment...,1jxt7ww,443,122,Sea_Contract3436,Not the A-hole,AITA for kicking out a homeless woman from the...,"So, I work at a 5\* hotel and more than a mont..."


In [15]:
df_clean = df[['id','flair','title','body']]



In [16]:
display(df_clean)

,id,flair,title,body
56,1jxvssj,Not the A-hole,AITA for not inviting one friend on our trip?,I've been friends with “Anna” and “Juan” since...
57,1jxvoyx,Not the A-hole,"AITA for the joke I made at a get together, in...",Istg I've got to stop giving my weekends away ...
58,1jxu9hu,Not the A-hole,AITA for not turning on Life 360 for my mom,I(19f) am a university student and I still li...
59,1jxtxfl,Asshole,AITA for bringing our daughter to my husband’s...,\n\nAITA for bringing our daughter to my husba...
60,1jxt7ww,Not the A-hole,AITA for kicking out a homeless woman from the...,"So, I work at a 5\* hotel and more than a mont..."
...,...,...,...,...
8787,1jojgxw,No A-holes here,AITA for telling my roommate to stop cooking f...,I (25F) asked my roommate (26M) to stop cookin...
8788,1jojau1,Not the A-hole,AITA for telling someone’s unattended child to...,"im 18f, have a 19 month old child, and i live ..."
8789,1joiw05,Not the A-hole,AITA For Not Helping My Parents When They Need Me,I (21) have a mom (39) who just had my little ...
8790,1joipct,Not the A-hole,AITA for leaving my boyfriends bday party afte...,went to his party made a proper effort to look...


In [17]:
df_clean['flair'].value_counts()

flair
Not the A-hole     5380
Asshole            1330
Everyone Sucks      460
No A-holes here     340
Not enough info     260
                     10
Name: count, dtype: int64

In [1]:
df_clean = df_clean.reset_index()

NameError: name 'df_clean' is not defined

In [ ]:
df_clean = df_clean.drop('index',axis=1)
display(df_clean.head())

,id,flair,title,body
0,1jxt7ww,Not the A-hole,AITA for kicking out a homeless woman from the...,"So, I work at a 5\* hotel and more than a mont..."
1,1jxt2o7,Not the A-hole,AITA for making a game based on an idea me and...,Basically back in 2021 me and a friend had an ...
2,1jxrssb,Asshole,AITA for preferring that my uncle teach me to ...,Throwaway because I know at least one of my si...
3,1jxrk9e,Not the A-hole,AITA for indirectly establishing a boundary wi...,I (F20) am currently in a situation where my m...
4,1jxr9tf,Not the A-hole,AITA for considering a business that competes ...,TL;DR: My friend runs a side hustle inspecting...


In [ ]:
df_clean.shape

(7780, 4)

In [ ]:
# df_clean.to_csv('aita7780.csv')